# **Pandas pour la Data Science**
## **Data cleaning : Nettoyage des Données et Gestion des NAs**

## **Introduction**

Le **nettoyage des données** et la bonne **gestion des valeurs manquantes** (appelées [NaN](https://en.wikipedia.org/wiki/NaN) ou **NA**) sont deux étapes essentielles avant toute analyse sur une base de données.

L'objectif de ce notebook est de détailler chacune de ces deux étapes afin d'obtenir un `DataFrame` propre et facilement exploitable.
En effet, les bases de données présentent très souvent ce genre de problèmes.

Pour cela, nous allons nous servir du `DataFrame` **`transactions`** importé dans l'exercice précédent.

- **(a)** Importer le module `pandas` sous le nom `pd` et charger le fichier `"transactions.csv"` dans le `DataFrame` **transactions**. Les données sont séparées par des **virgules** dans le ficher CSV et la colonne contenant les identifiants est **`'transaction_id'`**.
- **(b)** Afficher les 10 premières lignes de `transactions.csv` avec la méthode `head`.

In [2]:
# A
import pandas as pd

transactions = pd.read_csv("Transactions.csv", sep =',', index_col = "transaction_id")

# B
transactions.head(10)

,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type
transaction_id,,,,,,,,,
80712190438,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,e-Shop
29258453508,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,e-Shop
51750724947,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop
93274880719,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,e-Shop
51750724947,273420,23-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop
97439039119,272357,23-02-2014,8,3,-2,-824,173.040,-1821.040,TeleShop
45649838090,273667,22-02-2014,11,6,-1,-1450,152.250,-1602.250,e-Shop
22643667930,271489,22-02-2014,12,6,-1,-1225,128.625,-1353.625,TeleShop
79792372943,275108,22-02-2014,3,1,-3,-908,286.020,-3010.020,MBR


## **1. Nettoyage d'un jeu de données**

Dans cette partie nous allons introduire les méthodes de la classe `DataFrame` utiles au nettoyage d'un dataset. Ces méthodes peuvent se regrouper dans trois catégories différentes :

- **Gestion des doublons** (méthodes `duplicated` et `drop_duplicates`).
- **Modification des éléments** d'un `DataFrame` (méthodes `replace`, `rename` et `astype`).
- **Opérations** sur les valeurs d'un `DataFrame` (méthode `apply` et clause `lambda`).

### **Gestion des doublons (méthodes `duplicated` et `drop_duplicates`)**

Les **doublons** sont des entrées identiques qui apparaissent **plusieurs** fois dans un dataset.

Quand nous découvrons un jeu de données il est très important de vérifier **dès le départ** qu'il n'y ait pas de doublons.
La présence de doublons va générer des **erreurs** dans les calculs de statistiques ou le traçage de graphiques.

Soit **`df`** le `DataFrame` suivant :

| Nom    | Âge | Sexe | Taille |
|--------|-----|------|--------|
| Robert | 56  | M    | 174    |
| Mark   | 23  | M    | 182    |
| Alina  | 32  | F    | 169    |
| Mark   | 23  | M    | 182    |

La présence de doublons se vérifie à l'aide de la méthode **`duplicated`** d'un `DataFrame` :
```python
# On repère les lignes contenant des doublons
df.duplicated()

>>> 0  False
    1  False
    2  False
    3  True
```

Cette méthode renvoie un objet de la classe `Series` de `pandas`, équivalente à une colonne d'un `DataFrame`, qui nous dit pour chaque ligne si elle est un doublon.

Dans cet exemple, le résultat de la méthode `duplicated` nous informe que **la ligne d'indice 3 est un doublon**, c'est-à-dire que c'est la **copie exacte** d'une ligne précédente, dans ce cas la ligne 1.

Puisque la méthode `duplicated` nous renvoie un objet de la classe `Series`, nous pouvons lui appliquer la méthode **`sum`** pour compter le nombre de doublons :
```python
# Pour calculer la somme de booléens, on considère que True vaut 1 et False vaut 0.
print(df.duplicated().sum())
>>> 1
```
La méthode d'un `DataFrame` permettant de supprimer les doublons est **`drop_duplicates`**.
Son en-tête est la suivante :

`drop_duplicates(subset, keep, inplace)`

- Le paramètre `subset` indique la ou les colonnes à considérer pour identifier et supprimer les doublons. Par défaut, **`subset = None`** : on considère **toutes** les colonnes du `DataFrame`.
- Le paramètre `keep` indique quelle entrée doit être gardée :
  - **`'first'`** : On garde la **première** occurrence.
  - **`'last'`** : On garde la **dernière** occurrence.
  - **`'False'`** : On ne garde **aucune** des occurrences.
  - Par défaut, **`keep = 'first'`**.
- Le paramètre **`inplace`** (très courant dans les méthodes de la classe `DataFrame`), précise si l'on modifie **directement** le `DataFrame` (dans ce cas `inplace=True`) ou si la méthode renvoie une **copie** du `DataFrame` (`inplace=False`). Une méthode appliquée avec l'argument `inplace = True` est **irréversible**. Par défaut, `inplace = False`.

> Il faut être très prudent avec l'utilisation du paramètre `inplace`. Une bonne pratique est d'oublier ce paramètre et d'affecter le `DataFrame` retourné par la méthode à un **nouveau** `DataFrame`.

Le paramètre `keep` est celui qui est le plus souvent spécifié.
En effet, une base de données peut avoir des doublons créés à des dates différentes.
On spécifiera alors la valeur de l'argument `keep` pour ne garder que les entrées les plus récentes, par exemple.

Reprenons `DataFrame` `df` :

|        | Âge | Sexe | Taille |
|--------|-----|------|--------|
| Robert | 56  | M    | 174    |
| Mark   | 23  | M    | 182    |
| Alina  | 32  | F    | 169    |
| Mark   | 23  | M    | 182    |

Nous illustrons `df` avec la figure suivante :

![duplicates](https://github.com/diaBabPro/colabs/blob/main/duplicates.png?raw=true)

Nous illustrons dans les exemples suivants les entrées qui sont **supprimées** par la méthode `drop_duplicates` en fonction de la valeur du paramètre `keep` :
```python
# On ne garde que la première occurrence du doublon
df_first = df.drop_duplicates(keep = 'first')
```
![duplicates_first](https://github.com/diaBabPro/colabs/blob/main/duplicates_first.png?raw=true)

```python
# On ne garde que la dernière occurrence du doublon
df_last = df.drop_duplicates(keep = 'last')
```

![duplicates_last](https://github.com/diaBabPro/colabs/blob/main/duplicates_last.png?raw=true)

```python
# On ne garde aucun doublon
df_false = df.drop_duplicates(keep = False)
```

![duplicates_false](https://github.com/diaBabPro/colabs/blob/main/duplicates_false.png?raw=true)

- **(a)** Combien y a-t-il de doublons dans le `DataFrame` transactions ?


In [3]:
# A
transactions.duplicated().sum()

13

> Les transactions ont été enregistrées dans l'ordre antichronologique, c'est-à-dire que les **premières** lignes contiennent les transactions les plus récentes et les dernières lignes les transactions les plus anciennes.

- **(b)** Éliminer les doublons de la base de données en ne gardant que la première occurrence.
- **(c)** À l'aide des paramètres **`subset`** et **`keep`** de la méthode `drop_duplicates` de `transactions`, afficher la transaction **la plus récente** pour **chaque catégorie de `prod_cat_code`**. Pour cela, vous pourrez enlever tous les doublons de la colonne `prod_cat_code` en ne gardant que les premières occurrences.

In [4]:
# B
transactions_sans_doublons = transactions.drop_duplicates(keep = 'first')

# C
transactions_sans_doublons.drop_duplicates(subset = 'prod_cat_code', keep = 'first')

,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type
transaction_id,,,,,,,,,
80712190438,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,e-Shop
29258453508,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,e-Shop
51750724947,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop
93274880719,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,e-Shop
43134751727,268487,20-02-2014,3,2,-1,-611,64.155,-675.155,e-Shop
25963520987,274829,20-02-2014,4,4,3,502,158.130,1664.130,Flagship store


# **Modification des éléments d'un `DataFrame` (méthodes `replace`, `rename` et `astype`)**

La méthode **`replace`** permet de **remplacer** une ou plusieurs valeurs d'une colonne d'un `DataFrame`.

Son en-tête est le suivant :
```python
replace(to_replace, value, ...)
```
- Le paramètre `to_replace` contient la valeur ou la liste de valeurs **à remplacer**. Cela peut être une liste d'entiers, de chaînes de caractères, de booléens, etc...
- Le paramètre `value` contient la valeur ou la liste de valeurs **remplaçantes**. Cela peut aussi être une liste d'entiers, de chaines de caractères, de booléens, etc...

![replace](https://github.com/diaBabPro/colabs/blob/main/replace.png?raw=true)

En plus de modifier les éléments d'un `DataFrame`, il est possible de **renommer** ses colonnes.

Cela est possible grâce à la méthode **`rename`** qui prend en argument un **dictionnaire** dont les **clés** sont les **anciens** noms et les **valeurs** sont les **nouveaux** noms.
Il faut aussi renseigner l'argument **`axis = 1`** pour préciser que les noms à renommer sont ceux des colonnes.
```python
# Création du dictionnaire associant les anciens noms aux nouveaux noms de colonnes
dictionnaire = {'ancien_nom1': 'nouveau_nom1',
                'ancien_nom2': 'nouveau_nom2'}

# On renomme les variables grâce à la méthode rename
df = df.rename(dictionnaire, axis = 1)
```
Il est parfois nécessaire de modifier non seulement le nom d'une colonne mais aussi son **type**.

Par exemple, il se peut que lors de l'importation d'une base de données une variable soit de type chaîne de caractères alors qu'elle est réellement de type numérique.
Il suffit qu'une des entrées de la colonne soit mal reconnue et pandas considèrera que cette colonne est de type chaîne de caractères.

Cela est possible grâce à la méthode **`astype`**.

Les types que nous verrons le plus souvent sont :

- `str` : Chaîne de caractères (`'Bonjour'`).
- `float` : Nombre à virgule flottante (`1.0`, `1.14123`).
- `int` : Nombre entier (`1`, `1231`).
Comme pour la méthode **`rename`**, **`astype`** peut prendre en argument un dictionnaire dont les **clés** sont les **noms des colonnes** concernées et les **valeur**s sont les **nouveaux types** à assigner.
Cela est pratique si l'on veut modifier le type de plusieurs colonnes en même temps.

Le plus souvent, on voudra directement sélectionner la colonne dont on veut modifier le type et l'écraser en lui appliquant la méthode **`astype`**.
```python
# Méthode 1 : Création d'un dictionnaire puis appel à la méthode astype du DataFrame
dictionnaire = {'col_1': 'int',
                'col_2': 'float'}
df = df.astype(dictionnaire)

# Méthode 2 : Sélection de la colonne puis appel à la méthode astype d'une Series
df['col_1'] = df['col_1'].astype('int')
```
>   Les méthodes `rename` et `replace` disposent aussi du paramètre `inplace` pour effectuer l'opération directement sur le `DataFrame`. À utiliser avec grande prudence.

- Si vous vous trompez dans le prochain exercice, vous pouvez réimporter et effectuer le prétraitement des exercices précédents en lançant la cellule suivante :

In [5]:
# Importation des données
transactions = pd.read_csv("Transactions.csv", sep =',', index_col = "transaction_id")

# Suppression des doublons
transactions = transactions.drop_duplicates(keep = 'first')

- **(d)** Importer le module `numpy` sous le nom `np`.
- **(e)** Remplacer les modalités **`['e-Shop', 'TeleShop', 'MBR', 'Flagship store',  np.nan]`** de la colonne **`Store_type`** par les modalités **`[1, 2, 3, 4, 0]`**. On en profitera pour remplacer les nan de la colonne **`prod_subcat_code`**.
> La valeur `np.nanv est celle qui encode une valeur manquante. Nous allons remplacer cette valeur par `0`.
- **(f)** Convertir les colonnes **`Store_type`** et **`prod_subcat_code`** en type **`'int'`**.
- **(g)** Renommer les colonnes `Store_type`, `Qty`, `Rate` et `Tax` avec `store_type`, `qty`, `rate` et `tax`.

In [6]:
# D
import numpy as np

# E
transactions['Store_type'] = transactions['Store_type'].replace(['e-Shop', 'TeleShop', 'MBR', 'Flagship store', np.nan], [1, 2, 3, 4, 0])

# F
transactions['Store_type'] = transactions['Store_type'].astype('int')
transactions['prod_subcat_code'] = transactions['prod_subcat_code'].astype('int')

# G
new_names =  {'Store_type' : 'store_type',
              'Qty'        : 'qty',
              'Rate'       : 'rate',
              'Tax'        : 'tax'}
transactions = transactions.rename(new_names, axis = 1)

# Afficher les 15 premières lignes
transactions.head(15)

<ipython-input-6-672c610f6f41>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  transactions['Store_type'] = transactions['Store_type'].replace(['e-Shop', 'TeleShop', 'MBR', 'Flagship store', np.nan], [1, 2, 3, 4, 0])


,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
transaction_id,,,,,,,,,
80712190438,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,1
29258453508,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,1
51750724947,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,2
93274880719,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,1
51750724947,273420,23-02-2014,6,5,-2,-791,166.110,-1748.110,2
97439039119,272357,23-02-2014,8,3,-2,-824,173.040,-1821.040,2
45649838090,273667,22-02-2014,11,6,-1,-1450,152.250,-1602.250,1
22643667930,271489,22-02-2014,12,6,-1,-1225,128.625,-1353.625,2
79792372943,275108,22-02-2014,3,1,-3,-908,286.020,-3010.020,3


# **Opérations sur les valeurs d'un `DataFrame` (méthode `apply` et fonctions `lambda`)**

Il est souvent intéressant de modifier ou agréger les informations des colonnes d'un `DataFrame` à l'aide d'une opération ou d'une fonction.

Ces opérations peuvent être tout type de fonction **qui prend en argument une colonne**.
Ainsi, le module **numpy est parfaitement adapté** pour effectuer des opérations sur ce type d'objet.

La méthode permettant d'effectuer une opération sur une colonne est la méthode **`apply`** d'un `DataFrame` dont l'en-tête est :
```python
apply(func, axis, ...)
```
où :

- **`func`** est la fonction à appliquer à la colonne.
- **`axis`** est la dimension sur laquelle l'opération doit s'appliquer.

Exemple : `apply` et `np.sum`

Pour chaque colonne de type numérique, nous voulons calculer **la somme de toutes les lignes**.
La fonction `sum` de `numpy` effectue cette opération, ce qui nous permet de l'utiliser avec la méthode `apply`.

Puisque nous allons réaliser une opération sur les **lignes**, il faut donc préciser l'argument **`axis = 0`** dans la méthode `apply`.
```python
# Somme des lignes pour chaque COLONNE de df
 df_lines = df.apply(np.sum, axis = 0)
 ```
 Le résultat est le suivant :

![apply_sum_lines](https://github.com/diaBabPro/colabs/blob/main/apply_sum_lines.png?raw=true)

 Dans un second temps, nous voulons pour chaque ligne calculer **la somme de toutes les colonnes**.

Nous allons réaliser cette opération sur les colonnes, il faut donc préciser l'argument **`axis = 1`** dans la méthode `apply`.
```python
# Somme des colonnes pour chaque LIGNE de df
 df_columns = df.apply(np.sum, axis = 1)
 ```
 Le résultat est le suivant :

![apply_sum_columns](https://github.com/diaBabPro/colabs/blob/main/apply_sum_columns.png?raw=true)

 Ces exemples illustrent l'utilisation de la méthode `apply`.
Pour calculer une somme de lignes ou colonnes, il est préférable d'utiliser la méthode **`sum`** d'un `DataFrame` ou d'une `Series`, qui se comporte exactement de la même façon que la méthode `sum` d'un array numpy.

La colonne `tran_date` de `transactions` contient les dates des transactions au format **`('jour-mois-annee')`** (ex : `'28-02-2014'`).
Ces dates sont de type chaîne de caractères : Il est impossible d'effectuer des statistiques sur cette variable pour l'instant.

Nous voudrions plutôt avoir dans **3 colonnes différentes** les jours, mois et années de chaque transaction.
Ceci nous permettrait par exemple d'analyser et détecter des tendances dans les dates de transaction.

La date `'28-02-2014'` est une chaîne de caractères. Le jour, le mois et l'année sont séparées par un tiret `'-'`.
La classe des chaînes de caractères dispose de la méthode **`split`** pour découper une chaîne sur un caractère spécifique :
```python
date = '28-02-2014'

# Découpage de la chaîne sur le caractère '-'
print(date.split('-'))
>>> ['28', '02', '2014']
```
Cette méthode renvoie une **liste** contenant les découpes de la chaîne sur le caractère spécifié.
Ainsi, pour récupérer le jour, il suffit de sélectionner le **premier** élément du découpage. Pour récupérer le mois, il faut prendre le deuxième élément et pour l'année le troisième.

- **(h)** Définir une fonction **`get_day`** prenant en argument une chaîne de caractères et qui renvoie le premier élément de son découpage par le caractère `'-'`.
- **(i)** Définir les fonctions **`get_month`** et **`get_year`** qui font de même avec le deuxième et troisième élément du découpage.
- **(j)** Dans 3 variables **`days`**, **`months`** et **`years`**, stocker le résultat de la méthode **`apply`** sur la colonne **`tran_date`** appliquée avec les fonctions `get_day`, `get_month` et `get_year`. Comme ces fonctions s'appliquent élément par élément, il n'est pas nécessaire de spécifier l'argument **`axis`** dans la méthode `apply`.
- **(k)** Créer les colonnes `'day'`, `'month'` et `'year'` dans le `DataFrame` et y stocker les valeurs de `days`, `months` et `years`. La création d'une nouvelle colonne se fait simplement en la déclarant :
```python
# Création d'une nouvelle colonne 'day' avec les valeurs contenue dans days.
transactions['day'] = days
```
- **(l)** Afficher les 5 premières lignes de `transactions`.

In [7]:
# Split date
def get_element_in_date(date):
  return date.split('/')

# H
def get_day(date):
  return date.split('/')[0]

# I
def get_month(date):
  return date.split('/')[1]

def get_year(date):
  return date.split('/')[2]

# J

transactions['tran_date'] = transactions['tran_date'].str.replace('-', '/')

days = transactions['tran_date'].apply(get_day)
months = transactions['tran_date'].apply(get_month)
years = transactions['tran_date'].apply(get_year)

# K
transactions['day'] = days
transactions['month'] = months
transactions['year'] = years

# L
transactions.head(5)

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,day,month,year
transaction_id,,,,,,,,,,,,
80712190438,270351,28/02/2014,1,1,-5,-772,405.300,-4265.300,1,28,02,2014
29258453508,270384,27/02/2014,5,3,-5,-1497,785.925,-8270.925,1,27,02,2014
51750724947,273420,24/02/2014,6,5,-2,-791,166.110,-1748.110,2,24,02,2014
93274880719,271509,24/02/2014,11,6,-3,-1363,429.345,-4518.345,1,24,02,2014
51750724947,273420,23/02/2014,6,5,-2,-791,166.110,-1748.110,2,23,02,2014


La méthode **`apply`** est très puissante lorsqu'elle est associée à une fonction **`lambda`**.

En Python, le mot clé **`lambda`** est utilisé pour définir une fonction **anonyme** : une fonction déclarée sans nom.

Une fonction **`lambda`** peut prendre n'importe quel nombre d'arguments, mais ne peut avoir qu'une seule expression.

Voici sa syntaxe :
```python
lambda arguments: expression
```
Les fonctions `lambda` permettent donc de définir des fonctions avec une syntaxe très courte :
```python
# Exemple 1
x = lambda a: a + 2
print(x(3))
>>> 5
# Exemple 2
x = lambda a, b : a * b
print(x(2, 3))
>>> 6
# Exemple 3
x = lambda a, b, c : a - b + c
print(x(1, 2, 3))
>>> 2
```
Bien que syntaxiquement elles soient différentes, les fonctions **`lambda`** se comportent de la même manière que les fonctions régulières qui sont déclarées en utilisant le mot-clé **`def`**.

La définition classique d'une fonction se fait avec la clause **`def`** :
```python
def increment(x):
   return x+1
```
Il est aussi possible de définir une fonction avec la clause **`lambda`** :
```python
increment = lambda x: x+1
```
La première est très propre mais l'avantage de la seconde est de pouvoir être définie directement **au sein** de la méthode **`apply`**.

Ainsi, l'exercice précédent peut être fait avec une syntaxe très compacte :
```python
transactions['day'] = transactions['tran_date'].apply(lambda date: date.split('-')[0])
```
Ce genre de syntaxe est très pratique et très souvent utilisée pour le nettoyage de bases de données.


La colonne `prod_subcat_code` de `transactions` dépend de la colonne `prod_cat_code` car elle désigne une **sous-catégorie** de produit.
Il serait plus logique d'avoir la catégorie et sous-catégorie d'un produit dans la même variable.

Pour cela, nous allons fusionner les valeurs de ces deux colonnes :

- Nous allons d'abord convertir les valeurs de ces deux colonnes en chaîne de caractères à l'aide de la méthode **`astype`**.
- Ensuite, nous allons concaténer ces chaînes pour avoir un unique code représentant la catégorie et sous-catégorie. Ceci peut se faire de la façon suivante :
  ```python
  chaine1 = "Je pense"
  chaine2 = "donc je suis."

  # Concaténation des deux chaînes en les séparant par un espace
  print(chaine1 + " " + chaine2)
  >>> Je pense donc je suis.
  ```
Pour appliquer une fonction lambda à toutes les lignes, il faut spécifier l'argument **`axis = 1`** dans la méthode `apply`.
Dans la fonction elle-même, les colonnes de la ligne peuvent être accédées comme sur un `DataFrame` :
```python
# Calcul du prix unitaire d'un produit
transactions.apply(lambda row: row['total_amt'] / row['qty'], axis = 1)
```

- **(m)** À l'aide d'une fonction `lambda` appliquée sur `transactions`, créer une colonne **`'prod_cat'`** dans `transactions` contenant la concaténation des valeurs de `prod_cat_code` et `prod_subcat_code` séparées par un tiret `'-'`. N'oubliez pas de convertir les valeurs en chaînes de caractères.
L'affichage de cette colonne doit être le suivant :
```python
transaction_id
80712190438     1-1
29258453508     3-5
51750724947     5-6
93274880719    6-11
51750724947     5-6
             ...
94340757522    5-12
89780862956     1-4
85115299378     6-2
72870271171    5-11
77960931771    5-11
```

In [8]:
# M
transactions['prod_cat'] = transactions.apply(lambda row: str(row['prod_cat_code']) + '-' + str(row['prod_subcat_code']), axis = 1)

transactions.head(20)

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,day,month,year,prod_cat
transaction_id,,,,,,,,,,,,,
80712190438,270351,28/02/2014,1,1,-5,-772,405.300,-4265.300,1,28,02,2014,1-1
29258453508,270384,27/02/2014,5,3,-5,-1497,785.925,-8270.925,1,27,02,2014,3-5
51750724947,273420,24/02/2014,6,5,-2,-791,166.110,-1748.110,2,24,02,2014,5-6
93274880719,271509,24/02/2014,11,6,-3,-1363,429.345,-4518.345,1,24,02,2014,6-11
51750724947,273420,23/02/2014,6,5,-2,-791,166.110,-1748.110,2,23,02,2014,5-6
97439039119,272357,23/02/2014,8,3,-2,-824,173.040,-1821.040,2,23,02,2014,3-8
45649838090,273667,22/02/2014,11,6,-1,-1450,152.250,-1602.250,1,22,02,2014,6-11
22643667930,271489,22/02/2014,12,6,-1,-1225,128.625,-1353.625,2,22,02,2014,6-12
79792372943,275108,22/02/2014,3,1,-3,-908,286.020,-3010.020,3,22,02,2014,1-3


## **2. Gestion des valeurs manquantes**

Une **valeur manquante** est soit :

- Une valeur non renseignée.
- Une valeur qui n'existe pas. En général, elles sont issues de calculs mathématiques n'ayant pas de solution (une division par zéro par exemple).
Une valeur manquante apparaît sous la dénomination **NaN** ("**N**ot **a N**umber") dans un `DataFrame`.

Dans cette partie, nous allons voir plusieurs méthodes pour :

- La **détection** des valeurs manquantes (méthodes `isna` et `any`).
- Le **remplacement** de ces valeurs (méthode `fillna`).
- La **suppression** des valeurs manquantes (méthode `dropna`).
Dans un des exercices précédents, nous avons utilisé la méthode `replace` de `transactions` pour remplacer les valeurs manquantes par `0`.
Cette approche manque de rigueur et il ne faut pas procéder de cette façon en pratique.

Pour cette raison, nous allons réimporter la version brute de `transactions` pour annuler les étapes que nous avons faites dans les exercices précédents.

- **(a)** Lancer la cellule suivante pour réimporter `transactions`, enlever les doublons et renommer ses colonnes.

In [14]:
# Importation des données
transactions = pd.read_csv("Transactions.csv", sep =',', index_col = "transaction_id")

# Suppression des doublons
transactions = transactions.drop_duplicates(keep = 'first')

# Changement de nom des colonnes
new_names =  {'Store_type' : 'store_type',
              'Qty'        : 'qty',
              'Rate'       : 'rate',
              'Tax'        : 'tax'}

transactions = transactions.rename(new_names, axis = 1)

transactions.head()

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
transaction_id,,,,,,,,,
80712190438,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,e-Shop
29258453508,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,e-Shop
51750724947,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop
93274880719,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,e-Shop
51750724947,273420,23-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop


### **Détection des valeurs manquantes (méthodes `isna` et `any`)**

La méthode **`isna`** d'un `DataFrame` détecte ses valeurs manquantes. Cette méthode ne prend pas d'arguments.

Cette méthode retourne le même `DataFrame` dont les valeurs sont :

- **`True`** si la case du tableau originale est une valeur manquante (`np.nan`).
- **`False`** sinon.

>![is_null](https://github.com/diaBabPro/colabs/blob/main/is_null.png?raw=true)

Puisque la méthode `isna` renvoie un `DataFrame`, nous pouvons l'utiliser avec d'autres méthodes de la classe `DataFrame` pour avoir des informations plus précises :

- La méthode `any` avec son argument `axis` permet de déterminer **quelles colonnes** (`axis = 0`) ou **quelles lignes** (`axis = 1`) contiennent au moins une valeur manquante.
- La méthode `sum` compte le nombre de valeurs manquantes par colonne ou lignes (en spécifiant l'argument `axis`). Il est possible d'utiliser d'autres méthodes statistiques comme `mean`, `max`, `argmax`, etc...

Voici de nombreux exemples d'utilisation des méthodes `any` et `sum` avec `isna` :

On reprend le `DataFrame` **`df`** de l'illustration précédente :

|         | Nom       | Pays | Âge    |
|---------|-----------|------|--------|
| 0       | NaN       | Australie | NaN  |
| 1       | Duchamp   | France    | 25   |
| 2       | Hana      | Japon     | 54   |



L'instruction `df.isna()` renvoie :

|       | Nom   | Pays  | Âge   |
|-------|-------|-------|-------|
| 0     | True  | False | True  |
| 1     | False | False | False |
| 2     | False | False | False |

```python
# On détecte les COLONNES contenant au moins une valeur manquante
df.isna().any(axis = 0)

>>> Nom      True
    Pays     False
    Age      True
# On détecte les LIGNES contenant au moins une valeur manquante
df.isna().any(axis = 1)

>>> 0     True
    1    False
    2    False
# On utilise l'indexation conditionnelle pour afficher les entrées
# contenant des valeurs manquantes

df[df.isna().any(axis = 1)]
```
ce qui renvoie le `DataFrame` :

|       | Nom   | Pays      | Âge  |
|-------|-----------|------|------|
| 0     | NaN       | Australie | NaN  |

```python
# On compte le nombre de valeurs manquantes pour chaque COLONNE
df.isnull().sum(axis = 0) #Les fonctions isnull et isna sont strictement équivalentes

>>> Nom     1
    Pays    0
    Age     1
# On compte le nombre de valeurs manquantes pour chaque LIGNE
df.isnull().sum(axis = 1)

>>> 0    2
    1    0
    2    0
```

- **(b)** Combien de colonnes du `DataFrame` `transactions` contiennent des valeurs manquantes ?
- **(c)** Combien d'entrées de `transactions` contiennent des valeurs manquantes ? Vous pourrez suivre la méthode `any` avec la méthode `sum`.
- **(d)** Quelle colonne de `transactions` contient **le plus** de valeurs manquantes ?
- **(e)** Afficher les entrées de `transactions` qui contiennent au moins une valeur manquante dans les colonnes `'rate'`, `'tax'` et `'total_amt'`. Que remarquez-vous ?

In [32]:
# Importation des données (avec valeures manquantes)
transactions2 = pd.read_csv("Transactions2.csv", sep =',', index_col = "transaction_id")

# B
print(transactions2.isna().any(axis = 0))
print()
print(transactions2.isna().any(axis = 0).sum())

# C
print()
print(transactions2.isna().any(axis = 1).sum())

# D
print()
print(transactions2.isna().sum(axis = 0))

# E
print()
print(transactions2[transactions2[['Rate', 'Tax', 'total_amt']].isna().any(axis = 1)])

cust_id             False
tran_date           False
prod_subcat_code     True
prod_cat_code       False
Qty                  True
Rate                 True
Tax                  True
total_amt            True
Store_type           True
dtype: bool

6

7

cust_id             0
tran_date           0
prod_subcat_code    3
prod_cat_code       0
Qty                 2
Rate                2
Tax                 2
total_amt           2
Store_type          2
dtype: int64

                cust_id   tran_date  prod_subcat_code  prod_cat_code  Qty  \
transaction_id                                                              
51075317246      274987  19-02-2014              10.0              6  NaN   
63407734630      271322  19-02-2014               3.0              2  NaN   

                Rate  Tax  total_amt      Store_type  
transaction_id                                        
51075317246      NaN  NaN        NaN             MBR  
63407734630      NaN  NaN        NaN  Flagship store  


### **Remplacement des valeurs manquantes (méthode `fillna`)**

la méthode `fillna` permet de remplacer les valeurs manquantes d'un `DataFrame` par des valeurs de notre choix.
```python
# On remplace tous les NANs du DataFrame par des zéros
 df.fillna(0)

# On remplace les NANs de chaque colonne numérique par la moyenne sur cette colonne
 df.fillna(df.mean())  # df.mean() peut être remplacée par n'importe quelle méthode statistique.
 ```

 Il est courant de remplacer les valeurs manquantes d'une colonne de **type numérique** avec des **statistiques** comme :

>- La **moyenne** : `mean`.
>- La **médiane** : `median`.
>- Le **minimum/maximum** : `min`/`max`.

Pour les colonnes de type catégorielle, on remplacera les valeurs manquantes avec :

>- Le **mode**, i.e. la modalité la plus fréquente : `mode`.
>- Une **constante** ou catégorie arbitraire : `0`, `-1`.

Pour éviter de faire des erreurs de remplacement, il est fortement conseillé de **sélectionner les bonnes colonnes** avant d'utiliser la méthode `fillna`.

- Si vous faites des erreurs dans l'exercice suivant, vous pouvez réimporter `transactions` à l'aide de la cellule suivante :

In [ ]:
# Importation des données
transactions = pd.read_csv("transactions.csv", sep =',', index_col = "transaction_id")

# Suppression des doublons
transactions = transactions.drop_duplicates(keep = 'first')

# Changement de nom des colonnes
new_names =  {'Store_type' : 'store_type',
              'Qty'        : 'qty',
              'Rate'       : 'rate',
              'Tax'        : 'tax'}

transactions = transactions.rename(new_names, axis = 1)


- **(f)** Remplacer les valeurs manquantes de la colonne **`prod_subcat_code`** de `transactions` par `-1`.
- **(g)** Déterminer **la modalité la plus fréquente** (le mode) de la colonne **`store_type`** de transactions.
- **(h)** Remplacer les valeurs manquantes de la colonne `store_type` par cette modalité. On accède à la valeur de cette modalité **à l'indice 0** de la `Series` renvoyée par `mode`.
- **(i)** Vérifier que les colonnes `prod_subcat_code` et `store_type` de `transactions` ne contiennent plus de valeurs manquantes.

In [40]:
# F
transactions2['prod_subcat_code'] = transactions2['prod_subcat_code'].fillna(-1)
#print(transactions2.isna().any(axis = 0))

# G
print()
most_frequent_mode = transactions2['Store_type'].mode()[0]
print(most_frequent_mode)

# H
print()
transactions2['Store_type'] = transactions2['Store_type'].fillna(most_frequent_mode)

# I
print()
print(transactions2.isna().any(axis = 0))


e-Shop


cust_id             False
tran_date           False
prod_subcat_code    False
prod_cat_code       False
Qty                  True
Rate                 True
Tax                  True
total_amt            True
Store_type          False
dtype: bool


###**Suppression des valeurs manquantes (méthode `dropna`)**

La méthode `dropna` permet de supprimer les lignes ou colonnes contenant des valeurs manquantes.

L'en-tête de la méthode est la suivante : `dropna(axis, how, subset, ..)`

- Le paramètre **`axis`** précise si on doit supprimer des lignes ou des colonnes (**`0`** pour les lignes, **`1`** pour les colonnes).
- Le paramètre **`how`** permet de préciser comment les lignes (ou les colonnes) sont supprimées :
  - **`how = 'any'`**: On supprime la ligne (ou colonne) si elle contient **au moins** une valeur manquante.
  - **`how = 'all'`** : On supprime la ligne (ou colonne) si elle ne contient **que** des valeurs manquantes.
- Le paramètre **`subset`** permet de préciser les colonnes/lignes sur lesquelles on effectue la recherche de valeurs manquantes.

Exemple :

```python
# On supprime toutes les lignes contenant au moins une valeur manquante
df = df.dropna(axis = 0, how = 'any')

# On supprime les colonnes vides
df = df.dropna(axis = 1, how = 'all')

# On supprime les lignes ayant des valeurs manquantes dans les 3 colonnes 'col2','col3' et 'col4'
df = df.dropna(axis = 0, how = 'all', subset = ['col2','col3','col4'])
```

>Comme pour les autres méthodes de remplacement de valeurs d'un `DataFrame`, l'argument `inplace` peut être utilisé avec grande précaution pour effectuer la modification directement sans réassignation.

Les données de transactions pour lesquelles le montant de la transaction n'est pas renseigné ne nous sont pas intéressantes. Pour cette raison :

- **(j)** Supprimer les entrées de `transactions` pour lesquelles les colonnes **`rate`**, **`tax`** et **`total_amt`** sont **simultanément** vides.
- **(k)** Vérifier que les colonnes de `transactions` **ne contiennent plus** de valeurs manquantes.

In [41]:
# J
transactions2 = transactions2.dropna(axis = 0, how = 'all', subset = ['Rate', 'Tax', 'total_amt'])

# K
print(transactions2.isna().any(axis = 0))

cust_id             False
tran_date           False
prod_subcat_code    False
prod_cat_code       False
Qty                 False
Rate                False
Tax                 False
total_amt           False
Store_type          False
dtype: bool


# **Conclusion et récap**

Dans ce chapitre nous avons vu les méthodes essentielles du module `pandas` afin de nettoyer un dataset et gérer les valeurs manquantes (`NaN`).

Cette étape de préparation d'un dataset est **toujours** la première étape d'un projet data.

Concernant le **nettoyage des données**, nous avons ainsi appris à :

- Repérer et supprimer les doublons d'un `DataFrame` grâce aux méthodes **`duplicated`** et **`drop_duplicates`**.
- Modifier les éléments d'un `DataFrame` et leur type à l'aide des méthodes **`replace`**, **`rename`** et **`astype`**.
- Appliquer une fonction à un `DataFrame` avec la méthode **`apply`** et la clause **`lambda`**.

Concernant la **gestion des valeurs manquantes**, nous avons appris à :

- Les **détecter** grâce à la méthode **`isna`** suivie des méthodes **`any`** et **`sum`**.
- Les **remplacer** à l'aide de la méthode **`fillna`** et des **méthodes statistiques**.
- Les **supprimer** grâce à la méthode **`dropna`**.

Dans le notebook suivant, vous verrez d'autres manipulations de `DataFrame` pour une **exploration** des données plus avancées.